# Data Science Tools and Ecosystem

In this notebook, we will explore various tools, libraries, and concepts related to Data Science.

**Languages used in Data Science:**
- Python
- R
- SQL

**Common Data Science Libraries:**
- Pandas
- NumPy
- Matplotlib
- Scikit-learn

| Data Science Tool | Description |
|------------------|-------------|
| Jupyter Notebook | Interactive environment for coding |
| RStudio          | IDE for R programming              |
| Apache Spark     | Big data processing framework      |


Below are examples of simple arithmetic operations in Python.

In [ ]:
# This expression multiplies 3 by 4 and then adds 5
(3 * 4) + 5

In [ ]:
# Convert 200 minutes to hours by dividing by 60
minutes = 200
hours = minutes / 60
hours

**Objectives:**
- List popular languages used in Data Science
- List commonly used libraries in Data Science
- Demonstrate arithmetic expressions in Python

**Author:** Ahmed Fouda

**GitHub URL:** https://github.com/Eng-A7med2005/IBM